# Batch download place info from a csv file


In [ ]:
%load_ext autoreload
%autoreload 2

In [18]:
import csv

from loguru import logger as lg
from tqdm import tqdm
from requests import HTTPError

from trip_me_up.config.constants import DATA_FOL
from trip_me_up.config.credentials import PLACES_API_KEY
from trip_me_up.maps.place_api import FindPlace, PlaceDetails, PlaceDetailsFields

## Load csv data


In [ ]:
data_fn = "Da visitare.csv"
data_fp = DATA_FOL / data_fn
data_fp

In [ ]:
reader = csv.DictReader(data_fp.open())
place_names = [row["Titolo"] for row in reader]
place_names[0]

## Download place info


In [12]:
fp = FindPlace(PLACES_API_KEY)

In [13]:
pd = PlaceDetails(PLACES_API_KEY)

In [ ]:
pdf = PlaceDetailsFields(
    address_components=True,
    current_opening_hours=True,
    editorial_summary=True,
    formatted_address=True,
    formatted_phone_number=True,
    geometry=True,
    name=True,
    opening_hours=True,
    price_level=True,
    rating=True,
    types=True,
    user_ratings_total=True,
    website=True,
)
pdf.get_fields()

In [ ]:
place_details_all = []


for place_name in tqdm(place_names):
    # lg.info(f"Finding place_id for {place_name}")
    try:
        place_id = fp.find_place_id(place_name)
        if place_id is None:
            lg.warning(f"Could not find place_id for {place_name}")
            continue
        place_details = pd.get_place_details(place_id, pdf)
        place_details_all.append(place_details)
    except HTTPError as e:
        lg.error(f"Error for {place_name}: {e}")
        continue
    # print(place_details)
    # break

In [ ]:
place_details_all[5]["result"]["editorial_summary"]

In [ ]:
from collections import Counter


summary_counter = Counter(
    ["editorial_summary" in place["result"] for place in place_details_all]
)
summary_counter